# Data Collection

<hr>

Get an understanding of how [this](https://github.com/lmeazzini/youtube-recommender/blob/master/src/data_science/search_data_collection.py) works.

In [1]:
import time
import requests
import yaml

This isn't working :(

In [4]:
# open the .yaml file for parameters
with open('../parameters.yaml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)

# gets the parametesr from the .yaml file
queries = params['queries']
urls = params['urls']
number_of_pages = params['number_of_pages']

# Collecting data from youtube pages
for query in queries:
    for page in range(1, number_of_pages+1):
        url = urls.format(query=query, page=page)
        response = requests.get(url)

        with open('../../pages/{}_{}.html'.format(query,
                                                  page), 'w+') as output:
            output.write(response.text)

        time.sleep(1)

FileNotFoundError: [Errno 2] No such file or directory: '../../pages/folk+rock_1.html'

<hr>

New tutorial [here](https://www.youtube.com/watch?v=th5_9woFJmk). hehe

Load API key from the txt file. (There's probably a better way to do this, but oh well!)

In [2]:
API_KEY = ""

with open("../YouTube API Key.txt") as f:
    for line in f:
        API_KEY = line

Reference: https://github.com/googleapis/google-api-python-client  
Another reference: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.html

## Create the YouTube Service object

Get the YouTube service to let us work with the APIs

In [3]:
# Allows us to connect to the YouTube API service
from googleapiclient.discovery import build

# the service
youtube_service = build('youtube', 'v3', developerKey=API_KEY)

Simple API request as a test (from the tutorial video)

In [4]:
request = youtube_service.channels().list(
            part='statistics',
            forUsername='schafer5'
        )

In [5]:
response = request.execute()

In [6]:
print(response)

{'kind': 'youtube#channelListResponse', 'etag': 'lXXlwIGiu-wf7INz1lLPOZIg9Bo', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': '9ZvbmSCHow8TQjfuamaR6SnAnrA', 'id': 'UCCezIgC97PvUuR4_gbFUs5g', 'statistics': {'viewCount': '53160693', 'subscriberCount': '729000', 'hiddenSubscriberCount': False, 'videoCount': '230'}}]}


:)!

## List YouTube videos based off search keywords

*Now let's figure out how to search for YouTube music videos based on keywords.*

In [61]:
search_terms = 'study'
MUSIC_CATEGORY_ID = 10

In [62]:
request = youtube_service.search().list(
            part='snippet',
            eventType='completed',
            maxResults=50,
            q=search_terms,
            type='video',
            videoCategoryId = MUSIC_CATEGORY_ID,
            )

In [63]:
response = request.execute()

In [64]:
print(response)

{'kind': 'youtube#searchListResponse', 'etag': 'Zm7wDp9O9AcepaZosodjs6b0b_w', 'nextPageToken': 'CDIQAA', 'regionCode': 'US', 'pageInfo': {'totalResults': 266566, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#searchResult', 'etag': '344IeJ1euZVCzk89YsSx-8v0zro', 'id': {'kind': 'youtube#video', 'videoId': 'AFqv-dyuDtw'}, 'snippet': {'publishedAt': '2019-08-27T03:51:57Z', 'channelId': 'UCwobzUc3z-0PrFpoRxNszXQ', 'title': 'Study Music, Concentration, Focus, Meditation, Memory, Work Music, Relaxing Music, Study, ☯3596', 'description': 'Study Music, Concentration, Focus, Meditation, Memory, Work Music, Relaxing Music, Study, ☯3596 - Are you looking for studying music, focus music or ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/AFqv-dyuDtw/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/AFqv-dyuDtw/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/AFqv-dyuDtw/hqdefault.jpg', 'width': 480, 'height'

How to retrieve individual items (each item is a youtube video from the search)

In [65]:
response['items'][1]

{'kind': 'youtube#searchResult',
 'etag': '9-mK4cAIpWSe6Hz0fNj8BP2FTZI',
 'id': {'kind': 'youtube#video', 'videoId': 'Jhq_44IotCM'},
 'snippet': {'publishedAt': '2019-09-09T17:47:12Z',
  'channelId': 'UCwobzUc3z-0PrFpoRxNszXQ',
  'title': 'Study Music, Work Music, Focus, Concentration, Meditation, Relaxing Music, Calm Music, Study,\xa0☯3601',
  'description': 'Study Music, Work Music, Focus, Concentration, Meditation, Relaxing Music, Calm Music, Study, ☯3601 - Are you looking for studying music, focus music or ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/Jhq_44IotCM/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/Jhq_44IotCM/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/Jhq_44IotCM/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Yellow Brick Cinema - Relaxing Music',
  'liveBroadcastContent': 'none',
  'publishTime': '2019-09-09T17:47:12Z'}}

## Getting More Info Per Video (using the same API request as listing search results)  
Note after looking at stuff more: let's just use this API to get the list of video ID's, and then use the next one (`videos().list`) to get the actual info (because there's way more info available through that one, and I think it would be easier to do it all there)

Getting individual info on each item

How we get the <font color="red">youtube URL</font>

In [66]:
response['items'][1]['id']['videoId']

'Jhq_44IotCM'

In [67]:
print('https://www.youtube.com/watch?v='+response['items'][1]['id']['videoId'])

https://www.youtube.com/watch?v=Jhq_44IotCM


How to iterate through all the items **on the first page** 
(figure out how to iterate through pages next)  
<font color='red'>*Todo: edit this so it goes through more than just the first page*</font>

In [79]:
list_of_ids = []

for item in response['items']:
    list_of_ids.append(item['id']['videoId'])

list_of_ids

['AFqv-dyuDtw',
 'Jhq_44IotCM',
 'IjWfwkLSKtA',
 'fmOqGEis9Tw',
 '2CzAB9BNEAc',
 '2hWyiofjQXQ',
 'NQXMNzW-wu4',
 '77YwsoKsNV8',
 'XdDrv-8ZfwY',
 'Zh5vptlzKmY',
 'qf3W5ohfloo',
 'kCwxW7VH5mc',
 'xvj6IvhQSgE',
 '6obKeeH7cQc',
 'qY4uZy3HbOY',
 'fPV90bBtBqQ',
 'IF7UIruuBuU',
 'tUCs-Gdc63M',
 'pwApTgP8wg0',
 'GNsQkr_C4EQ',
 'jFwPLr7kOCI',
 'bFgpTDKj7eM',
 'sOG6xAhI4lA',
 '0kq20_7bJTo',
 'TCBhiCjdlaE',
 'B2ZrqeRCNR8',
 'sFcMCCC4sjw',
 'R2YeZCPK1y8',
 'OQCUNqECJuk',
 'Rwc7-xGTd9M',
 'SQ-XywbtJaE',
 'Vv25m72_xTA',
 'Uc_lHPnX76E',
 'rgrMNFn0Gi0',
 'gsjucPi-Mw4',
 'cTGOqdmVw3k',
 'n50_ZrAM884',
 '2JbEs2zvjZk',
 'HfUJC4R9his',
 'CFunfYKu1PY',
 'A6jzy4oK7KI',
 'wgt7BH5O4Ag',
 'dsjwbQURpMY',
 'bxNCJ2I0XT0',
 'I6_jiyX0z9M',
 'NoZ7I2L9b-k',
 'Lo4H13uAnXU',
 '6GeQuy534O8',
 'FgnT7lqO9GM',
 'Gg4P6HpKGIA']

To get the next page:

In [52]:
last_request = request
last_response = response

In [53]:
request = youtube_service.search().list_next(
                                        previous_request = last_request,
                                        previous_response = last_response)

In [54]:
response = request.execute()

In [60]:
for item in response['items']:
    print(item['snippet']['title'])
    print()

['study music', 'concentration music', 'focus music', 'study', 'relaxing music', 'studying music', 'focus', 'stress relief music', 'calm music', 'meditation music', 'alpha waves', 'background music', 'music for studying', 'brain power', 'calm music for studying', 'binaural beats', 'meditation', 'concentration', 'stress relief', 'work music', 'homework', 'intelligence', 'quarantine', 'memory', 'mindfulness', 'music for memory', 'relax', 'music for study work', 'music to study', 'zen music', 'yoga', 'yoga music', 'reading music', 'yellow brick cinema']



## More Info on Videos (new API request)

part can contain: contentDetails, fileDetails, id, liveStreamingDetails, localizations, player, processingDetails, recordingDetails, snippet, statistics, status, suggestions, topicDetails

See info here: https://developers.google.com/youtube/v3/docs/videos

<font color='red'>*Todo: look at what data here may be useful to save*</font>

In [80]:
request = youtube_service.videos().list(
            part='snippet',
            id=list_of_ids)

In [81]:
response = request.execute()

In [82]:
print(response)

{'kind': 'youtube#videoListResponse', 'etag': '5F5CTRPlFIQ7KFPf0tyKYTneHAQ', 'items': [{'kind': 'youtube#video', 'etag': 'kYmVesyhwTqKHAn5RHzh77j54uQ', 'id': 'AFqv-dyuDtw', 'snippet': {'publishedAt': '2019-08-27T03:51:57Z', 'channelId': 'UCwobzUc3z-0PrFpoRxNszXQ', 'title': 'Study Music, Concentration, Focus, Meditation, Memory, Work Music, Relaxing Music, Study, ☯3596', 'description': 'Study Music, Concentration, Focus, Meditation, Memory, Work Music, Relaxing Music, Study, ☯3596 - Are you looking for studying music, focus music or background music, embedded with alpha waves and binaural beats, to help with concentration, mindfulness, yoga, stress relief, homework and to improve your memory? Yellow Brick Cinema’s study music videos provide instrumental music for study to improve concentration and brain power. Would you like to try music to study? Our relaxing music for study, with its embedded binaural beats, will support your study sessions, helping you relax, work, study and focus yo

In [86]:
response['items'][0]['snippet']['title']

'Study Music, Concentration, Focus, Meditation, Memory, Work Music, Relaxing Music, Study, ☯3596'

In [87]:
response['items'][0]['snippet']['description']

'Study Music, Concentration, Focus, Meditation, Memory, Work Music, Relaxing Music, Study, ☯3596 - Are you looking for studying music, focus music or background music, embedded with alpha waves and binaural beats, to help with concentration, mindfulness, yoga, stress relief, homework and to improve your memory? Yellow Brick Cinema’s study music videos provide instrumental music for study to improve concentration and brain power. Would you like to try music to study? Our relaxing music for study, with its embedded binaural beats, will support your study sessions, helping you relax, work, study and focus your brain power. Indeed, this concentration music will keep your attention fully on your work and bring out your inner genius. Our work music can also be used as reading music, stress relief music, and focus music, allowing you to block out distractions and achieve stress relief. The binaural beats used in our calm music for studying make for excellent music for memory whilst working fr

In [89]:
response['items'][0]['snippet']['tags']

['study music',
 'concentration music',
 'focus music',
 'study',
 'relaxing music',
 'studying music',
 'focus',
 'stress relief music',
 'calm music',
 'meditation music',
 'alpha waves',
 'background music',
 'music for studying',
 'brain power',
 'calm music for studying',
 'binaural beats',
 'meditation',
 'concentration',
 'stress relief',
 'work music',
 'homework',
 'intelligence',
 'quarantine',
 'memory',
 'mindfulness',
 'music for memory',
 'relax',
 'music for study work',
 'music to study',
 'zen music',
 'yoga',
 'yoga music',
 'reading music',
 'yellow brick cinema']